In [ ]:
import pandas as pd
import os
!pip install chamd
import chamd


In [ ]:
for filename in os.listdir("/content/drive/MyDrive/LR_preds_FT/" ):
  try:
    df = pd.read_csv("/content/drive/MyDrive/LR_preds_FT/" + filename)
    end = filename.split("LOO")[1]
    number = end.split(".")[0]
    df.to_csv("/content/drive/MyDrive/LR_preds_FT/" + number + ".csv" )
  except:
    print("Something went wrong loading ")

NameError: ignored

In [ ]:
import random
runs = []
for idx in range(0,50):
  try:
    df = pd.read_csv("/content/drive/MyDrive/LR_PREDS_DB/" + str(idx) + ".csv")
    df["pred"][107]
    runs.append(df)
  except Exception:
    print("Something went wrong loading " + str(idx))
    runs.append(pd.DataFrame())

fileUrl = "/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/"
groups = ["cc", "cd"]
reader = chamd.ChatReader()
lines = []
for group in groups:
    for fileName in os.listdir(fileUrl + group):
        file = reader.read_file(fileUrl + group + "/" + fileName)
        lines_to_add = ""
        for line in file.lines:
            if line.metadata["speaker"].text == "PAR":
              lines_to_add = lines_to_add + line.text
        lines.append([lines_to_add, file.metadata["session"].text, 1 if group == "cc" else 0])

df = pd.DataFrame(lines, columns=["X", "session", "group"])
result_df = {}
for N in [5,15,25,35,45,1]:
    scores = []
    for run in range(0,100):
        selected_runs = []
        for i_ensemble in range(0, N):
            idx = random.randint(0, 49)
            if not runs[idx].empty:
              selected_runs.append(runs[idx])
        correct_votes = 0
        for df_idx in df.index:
            AD_votes = 0
            for selected_run in selected_runs:
                AD_votes = AD_votes + int(selected_run["pred"][df_idx])
            final_vote = 1 if AD_votes > (N - AD_votes) else 0
            correct_votes = (correct_votes + 1) if final_vote == [df["group"][df_idx]][0] else correct_votes
        scores.append(correct_votes / len(df.index))
    result_df[str(N)] = scores
pd.DataFrame(result_df).to_csv("/content/drive/MyDrive/DB_embeddings_lr.csv", index=True)

Something went wrong loading 0
Something went wrong loading 1
Something went wrong loading 2
Something went wrong loading 4
